In [1]:
!pip install transformers
import transformers

     |████████████████████████████████| 2.2MB 10.6MB/s 
     |████████████████████████████████| 3.3MB 42.1MB/s 
     |████████████████████████████████| 870kB 41.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=270b0924875183dcd7bf0f943409248698131026a1e222a3e79da061ed85d6bd
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


### Using pre-trained transformers
_for fun and profit_

There are many toolkits that let you access pre-trained transformer models, but the most powerful and convenient by far is [`huggingface/transformers`](https://github.com/huggingface/transformers). In this week's practice, you'll learn how to download, apply and modify pre-trained transformers for a range of tasks. Buckle up, we're going in!


__Pipelines:__ if all you want is to apply a pre-trained model, you can do that in one line of code using pipeline. Huggingface/transformers has a selection of pre-configured pipelines for masked language modelling, sentiment classification, question aswering, etc. ([see full list here](https://huggingface.co/transformers/main_classes/pipelines.html))

A typical pipeline includes:
* pre-processing, e.g. tokenization, subword segmentation
* a backbone model, e.g. bert finetuned for classification
* output post-processing

Let's see it in action:

In [2]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis', model="distilbert-base-uncased-finetuned-sst-2-english")

print(classifier("BERT is amazing!"))


[{'label': 'POSITIVE', 'score': 0.9998860955238342}]


In [3]:
import base64
data = {
    'arryn': 'As High as Honor.',
    'baratheon': 'Ours is the fury.',
    'stark': 'Winter is coming.',
    'tyrell': 'Growing strong.'
}

def predict(x):
  pred = classifier(x)[0]['label']
  if(pred == 'POSITIVE'):
    return True
  else:
    return False

data_keys = list(data.keys())

outputs = {data_keys[0]: predict(data_keys[0]), data_keys[1]: predict(data_keys[1]), 
           data_keys[2]: predict(data_keys[2]), data_keys[3]: predict(data_keys[3])}

assert sum(outputs.values()) == 2 and outputs[base64.decodestring(b'YmFyYXRoZW9u\n').decode()] == False
print("Well done!")

Well done!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: decodestring() is a deprecated alias since Python 3.1, use decodebytes()


You can also access vanilla Masked Language Model that was trained to predict masked words. Here's how:

In [4]:
mlm_model = pipeline('fill-mask', model="bert-base-uncased")
MASK = mlm_model.tokenizer.mask_token

for hypo in mlm_model(f"Donald {MASK} is the president of the united states."):
  print(f"P={hypo['score']:.5f}", hypo['sequence'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



P=0.99719 donald trump is the president of the united states.
P=0.00024 donald duck is the president of the united states.
P=0.00022 donald ross is the president of the united states.
P=0.00020 donald johnson is the president of the united states.
P=0.00018 donald wilson is the president of the united states.


In [5]:
# Your turn: use bert to recall what year was the Soviet Union founded in
for hypo in mlm_model(f"Solviet union was founded in year {MASK}."):
  print(f"P={hypo['score']:.5f}", hypo['sequence'])

P=0.03755 solviet union was founded in year 2000.
P=0.01763 solviet union was founded in year 2009.
P=0.01586 solviet union was founded in year 2008.
P=0.01577 solviet union was founded in year 2010.
P=0.01522 solviet union was founded in year 2001.


```

```

```

```


Huggingface offers hundreds of pre-trained models that specialize on different tasks. You can quickly find the model you need using [this list](https://huggingface.co/models).


In [6]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)


[{'word': 'Wolfgang', 'score': 0.9990139603614807, 'entity': 'B-PER', 'index': 4, 'start': 11, 'end': 19}, {'word': 'Berlin', 'score': 0.9996449947357178, 'entity': 'B-LOC', 'index': 9, 'start': 34, 'end': 40}]


In [7]:
text = """Almost two-thirds of the 1.5 million people who viewed this liveblog had Googled to discover
 the latest on the Rosetta mission. They were treated to this detailed account by the Guardian’s science editor,
 Ian Sample, and astronomy writer Stuart Clark of the moment scientists landed a robotic spacecraft on a comet 
 for the first time in history, and the delirious reaction it provoked at their headquarters in Germany.
  “We are there. We are sitting on the surface. Philae is talking to us,” said one scientist.
"""

# Task: create a pipeline for named entity recognition, use task name 'ner' and search for the right model in the list
ner_model = pipeline('ner', model='dslim/bert-base-NER')

named_entities = ner_model(text)
print(named_entities)

[{'word': 'Rose', 'score': 0.7991043329238892, 'entity': 'B-LOC', 'index': 27, 'start': 112, 'end': 116}, {'word': '##tta', 'score': 0.9511924982070923, 'entity': 'I-LOC', 'index': 28, 'start': 116, 'end': 119}, {'word': 'Guardian', 'score': 0.9982230067253113, 'entity': 'B-ORG', 'index': 40, 'start': 179, 'end': 187}, {'word': 'Ian', 'score': 0.9997612833976746, 'entity': 'B-PER', 'index': 46, 'start': 207, 'end': 210}, {'word': 'Sam', 'score': 0.9997870326042175, 'entity': 'I-PER', 'index': 47, 'start': 211, 'end': 214}, {'word': '##ple', 'score': 0.999646008014679, 'entity': 'I-PER', 'index': 48, 'start': 214, 'end': 217}, {'word': 'Stuart', 'score': 0.9997830986976624, 'entity': 'B-PER', 'index': 53, 'start': 240, 'end': 246}, {'word': 'Clark', 'score': 0.9997482299804688, 'entity': 'I-PER', 'index': 54, 'start': 247, 'end': 252}, {'word': 'Germany', 'score': 0.9997227191925049, 'entity': 'B-LOC', 'index': 85, 'start': 414, 'end': 421}, {'word': 'Phil', 'score': 0.9963127970695496,

In [8]:
print('OUTPUT:', named_entities)
word_to_entity = {item['word']: item['entity'] for item in named_entities}
assert 'org' in word_to_entity.get('Guardian').lower() and 'per' in word_to_entity.get('Stuart').lower()
print("All tests passed")

OUTPUT: [{'word': 'Rose', 'score': 0.7991043329238892, 'entity': 'B-LOC', 'index': 27, 'start': 112, 'end': 116}, {'word': '##tta', 'score': 0.9511924982070923, 'entity': 'I-LOC', 'index': 28, 'start': 116, 'end': 119}, {'word': 'Guardian', 'score': 0.9982230067253113, 'entity': 'B-ORG', 'index': 40, 'start': 179, 'end': 187}, {'word': 'Ian', 'score': 0.9997612833976746, 'entity': 'B-PER', 'index': 46, 'start': 207, 'end': 210}, {'word': 'Sam', 'score': 0.9997870326042175, 'entity': 'I-PER', 'index': 47, 'start': 211, 'end': 214}, {'word': '##ple', 'score': 0.999646008014679, 'entity': 'I-PER', 'index': 48, 'start': 214, 'end': 217}, {'word': 'Stuart', 'score': 0.9997830986976624, 'entity': 'B-PER', 'index': 53, 'start': 240, 'end': 246}, {'word': 'Clark', 'score': 0.9997482299804688, 'entity': 'I-PER', 'index': 54, 'start': 247, 'end': 252}, {'word': 'Germany', 'score': 0.9997227191925049, 'entity': 'B-LOC', 'index': 85, 'start': 414, 'end': 421}, {'word': 'Phil', 'score': 0.996312797

### The building blocks of a pipeline

Huggingface also allows you to access its pipelines on a lower level. There are two main abstractions for you:
* `Tokenizer` - converts from strings to token ids and back
* `Model` - a pytorch `nn.Module` with pre-trained weights

You can use such models as part of your regular pytorch code: insert is as a layer in your model, apply it to a batch of data, backpropagate, optimize, etc.

In [9]:
import torch
from transformers import AutoTokenizer, AutoModel, pipeline

model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


In [10]:
lines = [
    "Luke, I am your father.",
    "Life is what happens when you're busy making other plans.",
    ]

# tokenize a batch of inputs. "pt" means [p]y[t]orch tensors
tokens_info = tokenizer(lines, padding=True, truncation=True, return_tensors="pt")

for key in tokens_info:
    print(key, tokens_info[key])

print("Detokenized:")
for i in range(2):
    print(tokenizer.decode(tokens_info['input_ids'][i]))

input_ids tensor([[ 101, 5355, 1010, 1045, 2572, 2115, 2269, 1012,  102,    0,    0,    0,
            0,    0,    0],
        [ 101, 2166, 2003, 2054, 6433, 2043, 2017, 1005, 2128, 5697, 2437, 2060,
         3488, 1012,  102]])
token_type_ids tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
Detokenized:
[CLS] luke, i am your father. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] life is what happens when you're busy making other plans. [SEP]


In [11]:
# You can now apply the model to get embeddings
with torch.no_grad():
    token_embeddings, sentence_embedding = model(**tokens_info)

print(sentence_embedding)

pooler_output


### The search for similar questions.

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to **really** solve this task using context-aware embeddings.

In [ ]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

__Main task(3 pts):__ 
* Implement a function that takes a text string and finds top-k most similar questions from `quora.txt`
* Demonstrate your function using at least 5 examples

There are no prompts this time: you will have to write everything from scratch.

In [ ]:
!pip install sentence-transformers

In [3]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [26]:
sentences = []
for line in open('quora.txt'):
  sentences.append(line)

sentence_embeddings = sbert_model.encode(sentences)

"Can I get back with my ex even though she is pregnant with another guy's baby?\n"

In [41]:
from scipy.spatial import distance
import numpy as np

def topk_similar_questions(question, k):
  res = []
  sim_arr = []
  question_embedd = sbert_model.encode([question])[0]
  for sent_embedd in sentence_embeddings:
    sim = distance.cosine(question_embedd, sent_embedd)
    sim_arr.append(sim)
  sim_arr = np.array(sim_arr)
  indexes = sim_arr.argsort()[-k:][::-1]
  for index in indexes:
    res.append(sentences[index])
  return res

In [42]:
topk_similar_questions("Broke up with my ex", 3)

['Why are Golden Retriever dogs with blue eyes so active?\n',
 'What breed/breeds of dog is this?  Mixed breed with large upright ears and smooth fur.\n',
 'What are the ideal dimensions for a full screen ad on Android in landscape mode?\n']

```















```

__Bonus demo:__ transformer language models. 

`/* No points awarded for this task, but its really cool, we promise :) */`

In [ ]:
import torch
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', add_prefix_space=True)
model = GPT2LMHeadModel.from_pretrained('gpt2').train(False).to(device)

text = "The Fermi paradox "
tokens = tokenizer.encode(text)
num_steps = 1024
line_length, max_length = 0, 70

print(end=tokenizer.decode(tokens))

for i in range(num_steps):
    with torch.no_grad():
        logits = model(torch.as_tensor([tokens], device=device))[0]
    p_next = torch.softmax(logits[0, -1, :], dim=-1).data.cpu().numpy()

    next_token_index = p_next.argmax() #<YOUR CODE: REPLACE THIS LINE>
    # YOUR TASK: change the code so that it performs nucleus sampling

    tokens.append(int(next_token_index))
    print(end=tokenizer.decode(tokens[-1]))
    line_length += len(tokenizer.decode(tokens[-1]))
    if line_length >= max_length:
        line_length = 0
        print()



Transformers knowledge hub: https://huggingface.co/transformers/